# Forecasting to go further

In [ ]:
%matplotlib inline
import matplotlib as mpl
#from matplotlib import pylab as plot
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime as dt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

import collections

import pandas as pd
import numpy as np
#import tensorflow.compat.v2 as tf
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import bijectors as tfb

from tensorflow_probability import distributions as tfd
from tensorflow_probability import sts

#tf.enable_v2_behavior()

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

sns.set_context("notebook", font_scale=1.)
sns.set_style("whitegrid")
%config InlineBackend.figure_format = 'retina'

In [ ]:
raw_data = pd.read_csv('C:/Users/amaur/Documents/Data_science/Bayesian_TS/h1weekly.csv')
raw_data.head()

In [ ]:
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

# Calculate min and max date
min_date = raw_data['Date'].min()
max_date = raw_data['Date'].max()

# Generate weekly interval using pandas date_range
weekly_dates = pd.date_range(start=min_date, end=max_date, freq='W')

# Convert the pandas DatetimeIndex to a numpy.ndarray with datetime64 type
cancelation_dates = weekly_dates.to_numpy(dtype=np.datetime64)
cancelation_dates = cancelation_dates.astype('datetime64[W]')

## date printing formater 
cancelation_loc = mdates.DayLocator(interval=31)
cancelation_fmt = mdates.DateFormatter('%Y-%m')

## Number of forcasting steps (half a year)
num_forecast_steps = 26

## Formating time serie data
serial_data = raw_data.IsCanceled.tolist()
serial_data = np.array(serial_data).astype(np.float32)
## Training_data
training_data = serial_data[:-num_forecast_steps]

In [ ]:
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=62))
plt.plot(raw_data.Date, raw_data.IsCanceled)
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
indexed_raw_data = raw_data.set_index('Date')
indexed_raw_data.index.freq = 'W'
decomposition = seasonal_decompose(indexed_raw_data['IsCanceled'], model='additive')
plt.figure(figsize=(10, 8))
decomposition.plot()
plt.show()

In [ ]:
acf,confidence_interval=sm.tsa.acf(serial_data, nlags=20 ,alpha=0.05, fft=False)
plot_acf(serial_data, lags=20)

In [ ]:
pacf,confidence_interval=sm.tsa.pacf(serial_data, nlags=20 ,alpha=0.05)
plot_pacf(serial_data, lags=20)

In [ ]:
positive_bijector = tfb.Exp()
# Transform data for halfnormal bijection
approximate_unconstrained_rates_train = positive_bijector.inverse(tf.convert_to_tensor(training_data))
approximate_unconstrained_rates_serial = positive_bijector.inverse(tf.convert_to_tensor(serial_data))

In [ ]:
serial_data.std()

In [ ]:
num_timesteps = 89

In [ ]:
def build_model(approximate_unconstrained_rates, ar, ma):
  trend = sts.LocalLinearTrend(observed_time_series=approximate_unconstrained_rates)
  #seasonal_semester = tfp.sts.Seasonal(num_seasons=4, num_steps_per_season = 13, observed_time_series=observed_time_series,
  #name = 'semester variation')
  monthly_effect = tfp.sts.Seasonal(num_seasons=52, observed_time_series=approximate_unconstrained_rates,
  name = 'infra month variation')
  arma = tfp.sts.AutoregressiveIntegratedMovingAverage(ar_order=ar, ma_order=ma, observed_time_series=approximate_unconstrained_rates)
  return tfp.sts.Sum([trend, monthly_effect, arma], observed_time_series=approximate_unconstrained_rates)

In [ ]:
sts_model = build_model(approximate_unconstrained_rates_train, 3, 3)

In [ ]:
def sts_with_LogNormal_likelihood_model():
  # Encode the parameters of the STS model as random variables.
  param_vals = []
  for param in sts_model.parameters:
    param_val = yield param.prior
    param_vals.append(param_val)

  # Use the STS model to encode the log- (or inverse-softplus)
  # rate of a LogNormal.
  unconstrained_rate = yield sts_model.make_state_space_model(num_timesteps, param_vals)
  rate = positive_bijector.forward(unconstrained_rate[..., 0])
  observed_counts = yield tfd.LogNormal(loc = serial_data.mean(), scale=serial_data.std(), name='observed_cancelation')

model = tfd.JointDistributionCoroutineAutoBatched(sts_with_LogNormal_likelihood_model)

In [ ]:
pinned_model = model.experimental_pin(observed_cancelation=training_data)
constraining_bijector = pinned_model.experimental_default_event_space_bijector()

In [ ]:
surrogate_posterior = tfp.experimental.vi.build_factored_surrogate_posterior(event_shape=pinned_model.event_shape,
    bijector=constraining_bijector)

In [ ]:
pinned_model.event_shape

In [ ]:
# Allow external control of optimization to reduce test runtimes.
num_variational_steps = 200 # @param { isTemplate: true}
num_variational_steps = int(num_variational_steps)

losses = tfp.vi.fit_surrogate_posterior(pinned_model.unnormalized_log_prob,
                                        surrogate_posterior,
                                        optimizer=tf.optimizers.Adam(0.1),
                                        num_steps=num_variational_steps)

In [ ]:
plt.plot(losses)
plt.title("Variational loss")
_ = plt.xlabel("Steps")

In [ ]:
loc = np.log(serial_data.mean())

scale = np.log(serial_data.std())/2
scale

In [ ]:
def sample_forecasted_counts(sts_model, posterior_latent_rates,
                             posterior_params, num_steps_forecast,
                             num_sampled_forecasts):

  # Forecast the future latent unconstrained rates, given the inferred latent
  # unconstrained rates and parameters.
  unconstrained_rates_forecast_dist = tfp.sts.forecast(sts_model,
    observed_time_series=unconstrained_rate_samples,
    parameter_samples=posterior_params,
    num_steps_forecast=num_steps_forecast)

  # Transform the forecast to positive-valued Poisson rates.
  rates_forecast_dist = tfd.TransformedDistribution(
      unconstrained_rates_forecast_dist,
      positive_bijector)

  # Sample from the forecast model following the chain rule:
  # P(counts) = P(counts | latent_rates)P(latent_rates)
  sampled_latent_rates = rates_forecast_dist.sample(num_sampled_forecasts)
  sampled_forecast_counts = tfd.LogNormal(loc=sampled_latent_rates, scale = 0.001).sample()

  return sampled_forecast_counts, sampled_latent_rates

In [ ]:
posterior_samples = surrogate_posterior.sample(89)
param_samples = posterior_samples[:-1]
unconstrained_rate_samples = posterior_samples[-1][..., 0]
rate_samples = positive_bijector.forward(unconstrained_rate_samples)

plt.figure(figsize=(10, 4))
mean_lower, mean_upper = np.percentile(rate_samples, [10, 90], axis=0)
pred_lower, pred_upper = np.percentile(np.random.poisson(rate_samples), 
                                       [10, 90], axis=0)

_ = plt.plot(training_data, color="blue", ls='--', marker='o', label='observed', alpha=0.7)
_ = plt.plot(np.mean(rate_samples, axis=0), label='rate', color="green", ls='dashed', lw=2, alpha=0.7)
_ = plt.fill_between(np.arange(0, 89), mean_lower, mean_upper, color='green', alpha=0.2)
_ = plt.fill_between(np.arange(0, 89), pred_lower, pred_upper, color='grey', label='counts', alpha=0.2)
plt.xlabel("Day")
plt.ylabel("Daily Sample Size")
plt.title("Posterior Mean")
plt.legend()

In [ ]:
pred_lower, pred_upper = np.percentile(np.random.lognormal(mean = np.log(loc), sigma=np.log(scale)), 
                                       [10, 90], axis=0)

In [ ]:
forecast_samples, rate_samples = sample_forecasted_counts(
   sts_model,
   posterior_latent_rates=unconstrained_rate_samples,
   posterior_params=param_samples,
   # Days to forecast:
   num_steps_forecast=4,
   num_sampled_forecasts=50)

In [ ]:
forecast_samples = np.squeeze(forecast_samples)

In [ ]:
def plot_forecast_helper(data, forecast_samples, CI=90):
  """Plot the observed time series alongside the forecast."""
  plt.figure(figsize=(10, 4))
  forecast_median = np.exp(np.median(forecast_samples, axis=0))

  num_steps = len(data)
  num_steps_forecast = forecast_median.shape[-1]

  plt.plot(np.arange(num_steps), data, lw=2, color='blue', linestyle='--', marker='o',
           label='Observed Data', alpha=0.7)

  forecast_steps = np.arange(num_steps, num_steps+num_steps_forecast)

  CI_interval = [(100 - CI)/2, 100 - (100 - CI)/2]
  lower, upper = np.percentile(forecast_samples, CI_interval, axis=0)

  plt.plot(forecast_steps, forecast_median, lw=2, ls='--', marker='o', color='orange',
           label=str(CI) + '% Forecast Interval', alpha=0.7)
  plt.fill_between(forecast_steps,
                   lower,
                   upper, color='orange', alpha=0.2)

  plt.xlim([0, num_steps+num_steps_forecast])
  ymin, ymax = min(np.min(forecast_samples), np.min(data)), max(np.max(forecast_samples), np.max(data))
  yrange = ymax-ymin
  plt.title("{}".format('Observed time series with ' + str(num_steps_forecast) + ' Day Forecast'))
  plt.xlabel('Day')
  plt.ylabel('Daily Sample Size')
  plt.legend()

In [ ]:
np.median(forecast_samples, axis=0)

In [ ]:
plot_forecast_helper(training_data, forecast_samples)